In [21]:
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import telepot
from datetime import datetime
import pytz
import os
import numpy as np
import tempfile
import itertools as IT
# from google.colab.patches import cv2_imshow

In [22]:
def getTime():
  IST = pytz.timezone('Asia/Kolkata')
  timeNow = datetime.now(IST)
  return timeNow

In [23]:
# def print_results(video, limit=None):
def print_results(limit=None):
        trueCount = 0
        imageSaved = 0
        filename = 'savedImage.jpg'
        finalImage = 'finaImage.jpg'
        sendAlert = 0
        location = "Bangalore"
        #fig=plt.figure(figsize=(16, 30))

        print("Loading model ...")
        model = load_model('modelnew.h5')
        Q = deque(maxlen=128)
        vs = cv2.VideoCapture(0)
        writer = None
        (W, H) = (None, None)
        count = 0
        while True:
            # capture the datatime in string
            # print(type(str(datetime.now())))
            # break
            # read the next frame from the file
            (grabbed, frame) = vs.read()

            # if the frame was not grabbed, then we have reached the end
            # of the stream
            if not grabbed:
                break
            
            # if the frame dimensions are empty, grab them
            if W is None or H is None:
                (H, W) = frame.shape[:2]

            # clone the output frame, then convert it from BGR to RGB
            # ordering, resize the frame to a fixed 128x128, and then
            # perform mean subtraction
            
            output = frame.copy()
           
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (128, 128)).astype("float32")
            frame = frame.reshape(128, 128, 3) / 255

            # make predictions on the frame and then update the predictions
            # queue
            preds = model.predict(np.expand_dims(frame, axis=0))[0]
            # print("preds",preds)
            Q.append(preds)
            

            # perform prediction averaging over the current history of
            # previous predictions
            results = np.array(Q).mean(axis=0)
            i = (preds > 0.50)[0]
            label = i

            text_color = (0, 255, 0) # default : green

            if label: # Violence prob
                text_color = (0, 0, 255) # red
                trueCount = trueCount + 1

            else:
                text_color = (0, 255, 0)

            text = "Violence: {}".format(label)
            FONT = cv2.FONT_HERSHEY_SIMPLEX 

            cv2.putText(output, text, (35, 50), FONT,1.25, text_color, 3) 

            # check if the video writer is None
            if writer is None:
                # initialize our video writer
                IST = pytz.timezone('Asia/Kolkata')
                x = datetime.now(IST)
                x = 'recordedVideo'+str(x.strftime('%H%M%S%d%m%Y'))+'.mp4'
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter(x, fourcc, 60, (W, H), True)

            # write the output frame to disk
            writer.write(output)

            # show the output image
            cv2.imshow('', output)
            # cv2.waitKey(1)
            

            if(trueCount == 30):
            #   while imageSaved == 0:
              if(imageSaved == 0):
                if(label):
                  cv2.imwrite(filename, output)
                  imageSaved = 0
            # while sendAlert == 0:
              if(sendAlert == 0):
                timeMoment = getTime()
                bot = telepot.Bot('6189701593:AAGBccWBuVf9HGe2JS416p-TjU2AO07Mu0s')
                bot.sendMessage(-1001847057364, f"VIOLENCE ALERT!! \nLOCATION: {location} \nTIME: {timeMoment}")
                bot.sendPhoto(-1001847057364,  photo=open(filename, 'rb')) 
                sendAlert = 0
                # break
                trueCount = 0
            key = cv2.waitKey(1) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
        # release the file pointersq
        print("[INFO] cleaning up...")
        writer.release()
        vs.release()
        cv2.destroyAllWindows()

In [24]:
V_path = "V_19.mp4"  
NV_path = "NV_998.mp4"  

In [25]:
print_results()

Loading model ...
1/1 [==============================] - 0s 33ms/step
[INFO] cleaning up...
